Notebook de funciones que usan los demás notebooks. Este debe contener las fuciones actualizadas

# Procesamiento de datos

In [1]:
import pandas as pd
import numpy as np
import os
def raw_to_datafr (xlsPath, xlsPathMfgCurve):
    """

    :param xlsPath: path to data file
    :param xlsPathMfgCurve: path to manufacturer power curve file
    :return: processed dataframes
    """
    #imprimir a consola
    os.write(1, b"Inciando procesamiento de datos...\n")

    
    #xlsPath = 'C:/Users/mungu/Documents/DatosWTG.xlsx'
    #xlsPath = 'C:\\Users\\ernesto\\Dropbox\\Doctorado\\datos\\DatosWTG.xlsx'
    #xlsPathMfgCurve = 'Curva de potencia vestas 90.xlsx'

    #dataVPxls = pd.read_excel(xlsPath,usecols=[0,1,2],index_col=0,names=['vViento','Pacw'])
    dataVPxls = pd.read_excel(xlsPath,usecols=[0,1,2],index_col=0)
    dataVPxls.columns =['vViento','Pacw']
    #agrego la columna de potencia instantanea sin filtrar
    #dataVPxls['Pw']= (dataVPxls.iloc[1:,1].values-dataVPxls.iloc[0:-1,1]) * np.pi*45**2
    print('Total de registros: ' + str(len(dataVPxls)))
    #dfMfgCurve = pd.read_excel(xlsPathMfgCurve,usecols=[0,2],index_col=0,names=['pw'])#cambio esto en la nueva version
    dfMfgCurve = pd.read_excel(xlsPathMfgCurve,usecols=[0,2],index_col=0)
    dfMfgCurve.columns = ['pw']
    #marcando los datos faltantes asignando un nan a la fila completa
    datamk = dataVPxls
    datamk.loc[datamk.isnull().any(axis=1), :] = np.nan
    #numero de filas sin datos
    print('Numero de filas sin datos')
    print(datamk.loc[datamk.isnull().any(axis=1), :].isnull().sum())

    #eliminando filas con NaN. Si busco la fecha anterior debe aparecer error.
    cleanData = datamk.dropna()

    ###calculando la potencia
    #la pontencia del archivo de excel es la densidad de potencia acumulada.
    #Se tiene que hacer la resta de la potencia siguiente a la anterio y multiplicar por pi*r^2
    #el radio es 45m
    #si se hace la resta con un array de numpy (values) si se puede restar 
    #pues se hace elemento por elemento
    dataVP = cleanData.drop('Pacw',axis=1)
    dataVP['Pw']= (cleanData.iloc[1:,1].values-cleanData.iloc[0:-1,1]) * np.pi*45**2

    #eliminar el ultimo valor pues es NaN
    dataVP.drop(dataVP.tail(1).index,inplace=True) 

    #Eliminando outliers
    dataVP.drop([pd.to_datetime('2016-03-07 09:50:00')],inplace=True)
    #agregado para el reporte
    dataVP.drop([pd.to_datetime('2016-03-08 09:00:00')],inplace=True)
    dataVP.drop([pd.to_datetime('2016-01-02 06:40:00')],inplace=True)
    dataVP.drop([pd.to_datetime('2016-05-05 07:20:00')],inplace=True)
    dataVP.drop([pd.to_datetime('2016-05-23 23:00:00')],inplace=True)


    #leyendo la direccion del viento
    #dataDirVxls = pd.read_excel(xlsPath,sheet_name=1, usecols=[0,1],index_col=0,names=['Dir'])
    dataDirVxls = pd.read_excel(xlsPath,sheet_name=1, usecols=[0,1],index_col=0)
    dataDirVxls.columns =['Dir']
    #limpiando datos
    #marcando los datos faltantes asignando un nan a la fila completa
    datamkdir = dataDirVxls
    datamkdir.loc[datamkdir.isnull().any(axis=1), :] = np.nan
    #eliminando filas con NaN. Si busco la fecha anterior debe aparecer error.
    #tambien elimino el ultimo valor como lo hice en los datos de v y p
    #el copy es para que no me de la copy warning
    dataDir = datamkdir.dropna().copy()
    dataDir.drop(dataDir.tail(1).index,inplace=True) 
    #eliminando del outlier de viento misma fecha que el de la pontencia
    dataDir.drop([pd.to_datetime('2016-03-07 09:50:00')],inplace=True)
    #agregado para el reporte
    dataDir.drop([pd.to_datetime('2016-03-08 09:00:00')],inplace=True)
    dataDir.drop([pd.to_datetime('2016-01-02 06:40:00')],inplace=True)
    dataDir.drop([pd.to_datetime('2016-05-05 07:20:00')],inplace=True)
    dataDir.drop([pd.to_datetime('2016-05-23 23:00:00')],inplace=True)

    direcvrad= np.deg2rad(dataDir['Dir'].values)
    velocidades = dataVP.iloc[:]['vViento'].values
    vecVel = [-np.sin(direcvrad)*velocidades,np.cos(direcvrad)*velocidades]
    vecVelnp=np.array(vecVel).transpose()
    #original sin timestamp
    #dfVecVel = pd.DataFrame(data=vecVelnp,columns=['vx','vy']
    #con timestamp
    dfVecVel = pd.DataFrame(data=vecVelnp,columns=['vx','vy'],index=dataVP.index)

    #datos direccion velocidad
    #print(len(dataVP))
    #print(len(dataDir))
    #dataDV = pd.concat([dataDir,dataVP.vViento],axis=1)
    dataVDP = pd.concat([dataVP.vViento,dataDir,dataVP.Pw],axis=1)
    #dataVcD =pd.concat([dataDir,dfVecVel],axis=1)
    #Datos en p.u.. Divido todos los datos entre el máximo del conjunto de datos


    #OPERATIONAL DATA	
    # Rated power
    # 2,000 kW/2,200 kW
    # Cut-in wind speed	4 m/s
    # Cut-out wind speed	25 m/s
    # Re cut-in wind speed	23 m/s


    os.write(1, b"Fin del procesamiento de datos\n")
    return [dataVDP,dfVecVel,dfMfgCurve]


# Crear plot interactivo para cluster y subcluster

In [6]:
""'''
TODO:
    -que de una opción de mostrar solo viento o potencia aunque tenga subclusters
    -que funcione si solo incluyo dfclvv aunque tenga subclusters

'''
import matplotlib.pyplot as plt
from matplotlib.widgets import RadioButtons, TextBox
from IPython.display import display as wgdisplay
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interactive, fixed, interact_manual
import datetime
from matplotlib.ticker import EngFormatter
import matplotlib.patheffects as path_effects #efectos de texto
from matplotlib import font_manager
import matplotlib.lines as mlines
import seaborn as sns

sns.set()
plt.style.use('seaborn-white')
sns.set_style("whitegrid")

class PlotSubClusterInt:
    """
    Descripcion:
    Clase para dibujar dos subplots, uno vx vs vy y otro vviento vs Pw.

    Metodos:
        createPlot: Construye el plot de matplotlib.
            self.dfclvv: dataframe multiindice con dos columnas vx y vy agrupadas por cluster
            dfclpw: dataframe multiindice con dos columnas vviento y Pw agrupadas por cluster
            cl_method: string que almacena el metodo usado para hacer el cluster
                     |   C1    |   C2   |...
                    ----------------------
            Timestamp|  vx vy  |  vx vy |...            datavp: dataframe original de los datos de

        updatePlot: actuliza el plot en "tiempo real" segun los valores de los controles

        onClick: Es el la funcion que esta ligada al click en el plot. Solo ejecuta annotatePlot.

        annotatePlot: Crea una anotacion en el plot que indica donde el nombre del cluster mas cercano al conjunto de
              coordenadas donde se hizo click en el plot VP. Ademas muestra el cluster en el plot VV.

        blinkCluster: Resalta el cluster al que se hace referencia.

    Argumentos:
        cl_scl_order: el orden en que se hicieron los clusters (viento,viento), (viento,potencia),
            (potencia,viento),(potencia,potencia)
    """

    def __init__(self):
        self.axvp = None
        self.formatterPw = None
        self.fig = None
        self.axvv = None
        self.tbFilePath = None
        self.wchkcls = None
        self.tbreta = None
        self.wradText = None
        self.chkShowCPotFab = None
        self.chkShowBetz = None
        self.chkLimGlob = None
        self.chkShowCnt = None
        self.wdgPSize = None
        self.btnSelNoneChk = None
        self.btnSelAllChk = None
        self.btnUpdate = None
        self.vxmaxGlob = None
        self.pmaxGlob = None
        self.vvmaxGlob = None
        self.vvminGlob = None
        self.pminGlob = None
        self.vymaxGlob = None
        self.vyminGlob = None
        self.vxminGlob = None
        self.cl_avail = None
        self.clnames_all = None
        self.n_tot_clusters = None
        self.idx_centroids_sc = None
        self.idx_centroids = None
        self.dfclvp = None
        self.n_clusters = None
        self.showLegends = None
        self.savepath = None
        self.showlBetz = None
        self.showCent = None
        self.showMfgCurve = None
        self.showOpts = None
        self.vvento = None
        self.PMaxViento = None
        self.cl_scl_order = None
        from colorsys import hls_to_rgb

        self.fignum=999
        self.fisize = (5, 5)
        self.fontsize = 13
        self.labelFontSize = 13
        self.tickFontSize = 12
        self.markerSize = 100
        self.fontNameLabel = {'fontname':'Times New Roman'}
        self.fontNameCluster = {'fontname':'Arial'}
        self.ticks_font = font_manager.FontProperties(family='Times New Roman', style='normal',
                                                      size=self.labelFontSize, weight='normal', stretch='normal')
        self.n_subclu = 0
        colors1 = plt.cm.tab20(np.linspace(0., 1, 20))
        colors2 = plt.cm.Spectral(np.linspace(0, 1, 10))
        colorstab = np.vstack((colors1, colors2))
        self.mapa_colores=colorstab
        self.fisize = None
        self.dfclvv = None
        self.datavp = None
        self.dfMfgCurve=None
        self.dfclvp_is_empty = True


    def create_plot(self, dfclvv, dfclvp=None,datavp=None,figsize=(5, 5), cl_scl_order=(None, None),
                    idx_centroids=None,idx_centroids_sc=None,fign=999, savepath='', showCent =True,
                    showlBetz = True,showMfgCurve=True, showOpt = 'Magnitud',dfMfgCurve=None,
                    showLegends = True):
        """

        :param dfclvv:
        :param dfclvp:
        :param datavp:
        :param figsize:
        :param cl_scl_order:
        :param idx_centroids:
        :param idx_centroids_sc:
        :param fign:
        :param savepath:
        :param showCent:
        :param showlBetz:
        :param showMfgCurve:
        :param showOpt:
        :param dfMfgCurve:
        :param showLegends:
        """
        self.dfMfgCurve=dfMfgCurve
        if self.dfMfgCurve is None:
            os.write(1,b'Manufacturer power curve missing')
        self.showLegends = showLegends
        self.cl_scl_order = cl_scl_order
        self.dfclvp =dfclvp
        self.dfclvv = dfclvv
        self.datavp = datavp
        self.n_clusters = len(self.dfclvv.columns.levels[0])
        self.n_tot_clusters =self.n_clusters
        self.fignum =fign
        self.idx_centroids =idx_centroids
        self.idx_centroids_sc= idx_centroids_sc
        self.savepath = savepath
        self.showlBetz = showlBetz # show Betz's limit
        self.showCent = showCent # show clusters centroids
        self.showMfgCurve = showMfgCurve #show manufacturer's curve
        #which value is shown in the plot magnitude,cluster number,cluster name, none
        self.showOpts = showOpt

        #check if power data was passed
        if self.dfclvp is not None:
            self.dfclvp_is_empty = False

        ##################################
        #        limite de betz          #
        ##################################
        if not self.dfclvp_is_empty:
            A=np.pi*45**2
            Cp = 0.59 #limite de Betz
            rho = 1.1349
            self.vvento = np.unique(datavp.vViento.values)
            self.PMaxViento = 1/2*rho*A*self.vvento**3*Cp

        #########################################
        #           DATAFRAME ANALYSIS          #
        #########################################



        plt.ion()
        self.fig = plt.figure(self.fignum, figsize=self.fisize)
        self.axvv = self.fig.add_subplot(121)
        #1k, 1M
        self.formatterPw = EngFormatter(places=1, sep="\N{THIN SPACE}")  # U+2009
        # 3 because is cluster,subcluster,vx and vy
        if len(dfclvv.columns[0]) == 3:
            #last column has the last subcluster name SCn, this gives n
            self.n_subclu=int(dfclvv.columns[-1][1][2:])

        if not self.dfclvp_is_empty:
            if self.n_subclu > 0:  # existen subclusters , sino seria 2
                #numero total de clusters incluidos los subclusters
                self.n_tot_clusters = self.n_subclu*self.n_clusters
                # solo aplica cuando hay subclusters
                lev0 = self.dfclvv.columns.get_level_values(0)
                lev1 = self.dfclvp.columns.get_level_values(1)
                namcl = [(lev0[i],lev1[i]) for i in range(len(lev0))]
                #self.clnames_all = sorted(namcl[::2]) #todos los clusters y subclusters disponibles
                self.clnames_all = namcl[::2] #todos los clusters y subclusters disponibles
                #ordenar
                vv = [self.idx_centroids_sc.loc[c].vViento for c in self.clnames_all]
                idx = np.argsort(vv)
                x=[self.clnames_all[i] for i in idx]
                self.clnames_all=x.copy()
                del x
                #nombre de los clusters disponibles sin el nombre de los subclusters
                #self.cl_avail =sorted(set(item[0] for item in self.clnames_all))
                self.cl_avail =set(item[0] for item in self.clnames_all)
                #### BUSCAR MINIMOS Y MAXIMOS GLOBALES  (hacerlo más elegante)
                l=[self.dfclvv[cl].min() for cl in self.clnames_all]
                self.vxminGlob,self.vyminGlob =   np.amin(l,axis=0)
                l=[self.dfclvv[cl].max() for cl in self.clnames_all]
                self.vxmaxGlob,self.vymaxGlob =   np.amax(l,axis=0)

            else:  # no hay subcluster
                #lista ordenada con numeros y letras
                self.clnames_all=sorted(dfclvv.columns.levels[0],
                                        key=lambda x: int("".join([i for i in x if i.isdigit()])))
                #self.cl_avail =sorted(set(item for item in self.clnames_all))
                self.cl_avail =set(item for item in self.clnames_all)
            l=[self.dfclvp[cl].min() for cl in self.clnames_all]
            self.vvminGlob,self.pminGlob =   np.amin(l,axis=0)
            l=[self.dfclvp[cl].max() for cl in self.clnames_all]
            self.vvmaxGlob,self.pmaxGlob =   np.amax(l,axis=0)
        else:
            #lista ordenada con numeros y letras
            self.clnames_all=sorted(dfclvv.columns.levels[0],
                                    key=lambda x: int("".join([i for i in x if i.isdigit()])))
            #self.cl_avail =sorted(set(item for item in self.clnames_all))
            self.cl_avail =set(item for item in self.clnames_all)
            #### BUSCAR MINIMOS Y MAXIMOS GLOBALES  (hacerlo más elegante)
            l=[self.dfclvv[cl].min() for cl in self.clnames_all]
            self.vxminGlob,self.vyminGlob =   np.amin(l,axis=0)
            l=[self.dfclvv[cl].max() for cl in self.clnames_all]
            self.vxmaxGlob,self.vymaxGlob =   np.amax(l,axis=0)

        if not self.dfclvp_is_empty:
            self.axvp = self.fig.add_subplot(122)
            self.axvp.yaxis.set_major_formatter(self.formatterPw)



        ##############################################################################
        #                             WIDGETS                                        #
        ##############################################################################

        self.btnUpdate = widgets.Button(description='Actualizar')

        self.btnUpdate.on_click(self.update_plot)
        self.btnSelAllChk = widgets.Button(description='Sel. todo')
        self.btnSelAllChk.on_click(self.sel_all_chk)
        self.btnSelNoneChk = widgets.Button(description='Des. todo')
        self.btnSelNoneChk.on_click(self.sel_none_chk)
        self.wdgPSize = widgets.IntSlider(
            value=2,
            min=1,
            max=20,
            step=1,
            description='Point size:',
            continuous_update=False)
        self.chkLimGlob = widgets.Checkbox(
            value=True, description='Límites Globales')
        self.chkShowCnt = widgets.Checkbox(
            value=self.showCent, description='Mostrar centroides')
        self.chkShowBetz = widgets.Checkbox(
            value=self.showlBetz, description='Mostrar línea Betz')
        self.chkShowCPotFab = widgets.Checkbox(
            value=self.showMfgCurve, description='Mostrar curva Fab.')
        if not self.dfclvp_is_empty:
            if self.dfMfgCurve is not None:
                self.chkShowCPotFab.disabled= False
            else:
                self.chkShowCPotFab.disabled= True
        else:
            self.chkShowBetz.disabled=True


        self.tbreta = widgets.IntText(
            value=0, description='Retardo:', layout=Layout(width='90%', height='80px'))
        self.wradText = widgets.RadioButtons(
            options=['Magnitud', 'Numero', 'Clusters', 'Ninguno'],
            description='Mostrar texto:')
        self.wradText.value= self.showOpts
        figsavepath = self.savepath
        figsavename = 'clustersplot'+str(self.n_clusters)+'SCl'+str(self.n_subclu)+'_'
        figsavetime = datetime.datetime.now().strftime("%d-%m-%Y_%H_%M_%S_%f")
        self.tbFilePath = widgets.Text(
            value=figsavepath+figsavename+figsavetime+'.jpg',
            description='Save path:', layout=Layout(width='90%', height='80px'))
        #self.chkShowCnt.observe(self.updatePlot, 'value')
        #self.wdgPSize.observe(self.updatePlot, 'value')
        self.wradText.observe(self.update_plot, 'value')
        #self.chkShowBetz.observe(self.updatePlot, 'value')
        #self.chkShowCPotFab.observe(self.updatePlot, 'value')


        self.wchkcls=[] #lista de checbox con los nombresde los clusters
        n=1#para numerar la lista de clusters
        for i in range(len(self.clnames_all)):
            self.wchkcls.append(widgets.Checkbox(
                value=True, description=str(n) +'-' +str(self.clnames_all[i])))
            n+=1
        box_layout = Layout(display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            height='200px',
                            )
        vbchkcls = widgets.VBox(self.wchkcls, layout=box_layout)

        self.text_log=widgets.Text(
            value='',
            placeholder='Log',
            disabled=False
        )

        # -------------   WIDGETS EN CAJAS ----------------------
        vbopt1 = widgets.VBox([self.chkShowCnt,self.chkShowBetz, self.chkShowCPotFab,self.chkLimGlob, self.tbreta])
        vbopt2 = widgets.VBox([self.wradText, self.wdgPSize ])
        vbButtons = widgets.VBox([self.btnSelAllChk,self.btnSelNoneChk, self.btnUpdate, self.tbFilePath])
        vblog = widgets.VBox([self.text_log])
        items = [vbButtons, vbchkcls,vbopt1,vbopt2]
        hb = widgets.HBox(items)
        wgdisplay(hb)
        wgdisplay(vblog)
        #wgdisplay(self.fig)
        #####################################
        #               PLOTs               #
        #####################################

        self.update_plot(1)

    def sel_all_chk(self,val):
        """

        :param self:
        :param val:
        """
        for item in self.wchkcls:
            item.value=True
    def sel_none_chk(self,val):
        """

        :param self:
        :param val:
        """
        for item in self.wchkcls:
            item.value=False
    def save_plot(self,val):
        """

        :param self:
        :param val:
        """
        self.text_log.value ='Guardando plot...'
        figsavepath = self.savepath
        figsavename = 'VVVPCl'+str(self.n_clusters)+'SCl'+str(self.n_subclu)+'_'
        #         figsavetime = datetime.datetime.now().strftime("%d-%m-%Y_%H_%M_%S_%f")
        self.tbFilePath.value=figsavepath+figsavename+'.png'

        plt.savefig(  self.tbFilePath.value, bbox_inches='tight', pad_inches=0.1)
        #print('Saved in ' +  self.tbFilePath.value)
        self.text_log.value ='Plot guardado en: ' + self.tbFilePath.value

    def update_plot(self, val):
        """
        :param self:
        :param val:
        """
        self.text_log.value = 'Actualizando plot...'
        pSize = self.wdgPSize.value
        self.axvv.axes.clear()
        self.axvv.grid(visible = True)
        if not self.dfclvp_is_empty:
            self.axvp.axes.clear()
            self.axvp.grid()


        if self.n_subclu>0 :  # existen subclusters

            clnames= [eval(el.description.split('-')[1]) for el in self.wchkcls if el.value==True]
            #ordenar subclusters por velocidad
            #los subclusters se ordenan dentro de cada grupo de clusters en idx_centrods_sc
            #hay que ordenarlos tambien globalmente y no solo localmente
            #self.cl_avail =sorted(set(item[0] for item in clnames))
            vv = [self.idx_centroids_sc.loc[c].vViento for c in clnames]
            idx = np.argsort(vv)
            x=[clnames[i] for i in idx]
            clnames=x.copy()
            del x
            #self.clnames_all =clnames# REV: SE OCUPA CLNAMES_ALL???
            self.cl_avail =set(item[0] for item in clnames)

        else :
            clnames=[el.description.split('-')[1] for el in self.wchkcls if el.value==True]
            #aqui tambien va por si se eliminan todos los subclusters del mismo cluster. Solo lista los clusters
            #self.cl_avail =sorted(set(item for item in clnames))
            self.cl_avail =set(item for item in clnames)


        #------------ DEFINIR LIMITES DE PLOT ---------------------

        #buscar los minimos y maximos de los clusters
        #debede haber una forma mas elegante de hacerlo. Como hago sliced elmultiindex con tuplas

        lvxmin=np.empty(self.n_tot_clusters)
        lvxmin.fill(np.nan)
        lvymin=np.empty(self.n_tot_clusters)
        lvymin.fill(np.nan)
        lvxmax=np.empty(self.n_tot_clusters)
        lvxmax.fill(np.nan)
        lvymax=np.empty(self.n_tot_clusters)
        lvymax.fill(np.nan)
        lvvmin=np.empty(self.n_tot_clusters)
        lvvmin.fill(np.nan)
        lvvmax=np.empty(self.n_tot_clusters)
        lvvmax.fill(np.nan)
        lpmin =np.empty(self.n_tot_clusters)
        lpmin.fill(np.nan)
        lpmax= np.empty(self.n_tot_clusters)
        lpmax.fill(np.nan)
        n=0
        for cl in clnames:
            lvxmin[n],lvymin[n]= self.dfclvv[cl].min()
            lvxmax[n],lvymax[n] = self.dfclvv[cl].max()
            if not self.dfclvp_is_empty:
                lvvmin[n],lpmin[n] = self.dfclvp[cl].min()
                lvvmax[n],lpmax[n] = self.dfclvp[cl].max()

            n+=1
        vxmin=np.nanmin(lvxmin)
        if np.isnan(vxmin):
            vxmin=0
        vymin = np.nanmin(lvymin)
        if np.isnan(vymin):
            vymin =0
        vxmax = np.nanmax(lvxmax)
        if np.isnan(vxmax):
            vxmax=1
        vymax = np.nanmax(lvymax)
        if np.isnan(vymax):
            vymax=1

        if not self.dfclvp_is_empty:
            vvmin = np.nanmin(lvvmin)
            if np.isnan(vvmin):
                vvmin=0
            vvmax = np.nanmax(lvvmax)
            if np.isnan(vvmax):
                vvmax=1

            pmin= np.nanmin(lpmin)
            if np.isnan(pmin):
                pmin=0
            pmax = np.nanmax(lpmax)
            if np.isnan(pmax):
                pmax=1

        if self.chkLimGlob.value:#plotear con limites globales o con los limites del cluster
            self.axvv.set_xlim((self.vxminGlob, self.vxmaxGlob))
            self.axvv.set_ylim((self.vyminGlob, self.vymaxGlob))
            if not self.dfclvp_is_empty:
                self.axvp.set_xlim((self.vvminGlob, self.vvmaxGlob))
                self.axvp.set_ylim((self.pminGlob, self.pmaxGlob))
        else:
            self.axvv.set_xlim((vxmin, vxmax))
            self.axvv.set_ylim((vymin, vymax))
            if not self.dfclvp_is_empty:
                self.axvp.set_xlim((vvmin, vvmax))
                self.axvp.set_ylim((pmin, pmax))

        #####################################
        #               PLOTEAR             #
        #####################################
        for item in clnames:
            #busca el cluster actual y devuelve el indice dentro de la lista de clusters donde lo encuentra
            #es decir, asocia un numero unico a un nombre de cluster
            #es para que el color de los clusters sea el mismo siempre
            idxClName = [ncl_ for ncl_, clname_ in enumerate(self.clnames_all) if clname_ == item]
            #self.fig.suptitle('Grupos de velocidad de viento y potencia', y=1)
            # magnitud del vector

            magni = round(
                np.mean(
                    np.sqrt(self.dfclvv[item].vx**2 +
                            self.dfclvv[item].vy**2)),
                1)  # magnitud de la vv

            self.axvv.scatter(
                self.dfclvv[item].vx,
                self.dfclvv[item].vy,
                s=pSize,
                c=self.mapa_colores[idxClName],
                alpha=1)

#------------------------MOSTRAR TEXTO -----------------------------
            if self.wradText.index == 0: #magnitud

                text = self.axvv.text(
                    self.dfclvv[item].vx.mean(),
                    self.dfclvv[item].vy.mean(),
                    magni,
                    fontsize=self.fontsize,
                    weight='bold',
                    color='w',
                    alpha=1,
                    zorder=100,
                    **self.fontNameCluster
                )
                text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='k'),
                                       path_effects.Normal()])

                if not self.dfclvp_is_empty:
                    text = self.axvp.text(
                        self.dfclvp[item].vViento.mean(),
                        self.dfclvp[item].Pw.mean(),
                        magni,
                        fontsize=self.fontsize,
                        weight='bold',
                        color='w',
                        alpha=1,
                        zorder=100,
                        **self.fontNameCluster
                    )
                    text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='k'),
                                           path_effects.Normal()])

            elif self.wradText.index == 1: #numero
                text = self.axvv.text(
                    self.dfclvv[item].vx.mean(),
                    self.dfclvv[item].vy.mean(),
                    idxClName[0]+1,
                    fontsize=self.fontsize,
                    weight='bold',
                    color='w',
                    alpha=1,
                    zorder=100,
                    **self.fontNameCluster
                )
                text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='k'),
                                       path_effects.Normal()])

                if not self.dfclvp_is_empty:
                    text = self.axvp.text(
                        self.dfclvp[item].vViento.mean(),
                        self.dfclvp[item].Pw.mean(),
                        idxClName[0]+1,
                        fontsize=self.fontsize,
                        weight='bold',
                        color='w',
                        alpha=1,
                        zorder=100,
                        **self.fontNameCluster
                    )
                    text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='k'),
                                           path_effects.Normal()])

            elif self.wradText.index == 2:
                text = self.axvv.text(
                    self.dfclvv[item].vx.mean(),
                    self.dfclvv[item].vy.mean(),
                    item,
                    fontsize=self.fontsize,
                    weight='bold',
                    color='w',
                    alpha=1,
                    zorder=100,
                    **self.fontNameCluster
                )
                text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='k'),
                                       path_effects.Normal()])

                if not self.dfclvp_is_empty:
                    text = self.axvp.text(
                        self.dfclvp[item].vViento.mean(),
                        self.dfclvp[item].Pw.mean(),
                        item,
                        fontsize=self.fontsize,
                        weight='bold',
                        color='w',
                        alpha=1,
                        zorder=100,
                        **self.fontNameCluster
                    )
                    text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='k'),
                                           path_effects.Normal()])

        ######### MOSTRAR CENTROIDES#######################
        if self.chkShowCnt.value:

            for cl in self.cl_avail:

                #el nombred el cluster esta en el indice
                #le quito la letra con cl[1:] y dejo solo el numero como
                #esta en el dataframe
                numcl = int(cl[1:])
                self.axvv.scatter(
                    self.idx_centroids.loc[numcl].vx,
                    self.idx_centroids.loc[numcl].vy,
                    marker='X',
                    edgecolor='black',
                    linewidth=1,
                    facecolor='yellow',
                    s=self.markerSize)
                if not self.dfclvp_is_empty:
                    self.axvp.scatter(
                        self.idx_centroids.loc[numcl].vViento,
                        self.idx_centroids.loc[numcl].Pw,
                        marker='X',
                        edgecolor='black',
                        linewidth=1,
                        facecolor='yellow',
                        s=self.markerSize)

            ####### MOSTRAR CENTROIDES SUBCLUSTERS
            if self.n_subclu > 0:
                for el in clnames:
                    self.axvv.scatter(
                        self.idx_centroids_sc.loc[el].vx,
                        self.idx_centroids_sc.loc[el].vy,
                        marker='h',
                        edgecolor='black',
                        linewidth=1,
                        facecolor='aqua',
                        s=self.markerSize)
                    if not self.dfclvp_is_empty:
                        self.axvp.scatter(
                            self.idx_centroids_sc.loc[el].vViento,
                            self.idx_centroids_sc.loc[el].Pw,
                            marker='h',
                            edgecolor='black',
                            linewidth=1,
                            facecolor='aqua',
                            s=self.markerSize)

        #MOSTRAR CURVA DEL FABRICANTE
        if not self.dfclvp_is_empty:
            if self.chkShowCPotFab.value:
                self.axvp.plot(self.dfMfgCurve.index, self.dfMfgCurve.pw, c='red', label='Manufacturer')

        #MOSTRAR LIMITE DE BETZ
        if not self.dfclvp_is_empty:
            if self.chkShowBetz.value:
                self.axvp.plot(self.vvento,self.PMaxViento,label='Betz',c='y')

        if not self.dfclvp_is_empty:
            self.axvp.set_xlabel('Wind Speed [m/s]',fontsize = self.labelFontSize, **self.fontNameLabel)
            self.axvp.set_ylabel('Power [W]',fontsize = self.labelFontSize, **self.fontNameLabel)
            self.axvp.tick_params(axis='both', which='major')

        self.axvv.set_xlabel('vx [m/s]',fontsize = self.labelFontSize, **self.fontNameLabel)
        self.axvv.set_ylabel('vy [m/s]',fontsize = self.labelFontSize, **self.fontNameLabel)
        self.axvv.tick_params(axis='both', which='major')

        #con esto cambio el texto de las thicks a el que defino en self.tick_font
        for label in self.axvv.get_xticklabels():
            label.set_fontproperties(self.ticks_font)
        for label in self.axvv.get_yticklabels():
            label.set_fontproperties(self.ticks_font)

        if not self.dfclvp_is_empty:
            for label in self.axvp.get_xticklabels():
                label.set_fontproperties(self.ticks_font)
            for label in self.axvp.get_yticklabels():
                label.set_fontproperties(self.ticks_font)
        #MOSTRAR LEYENDA

        if self.showLegends:
            legCentroids = mlines.Line2D([], [], color='yellow', marker='X', linestyle='None',
                                         markersize=10, label='Centroids',markeredgecolor='black',markeredgewidth=1.5)


            if self.n_subclu > 0:
                legSecCentroids = mlines.Line2D([], [], color='aqua', marker='h', linestyle='None',
                                                markersize=10, label='Sec. centroids',markeredgecolor='black',markeredgewidth=1)
                if not self.dfclvp_is_empty:
                    self.axvp.legend(handles=[legCentroids, legSecCentroids],facecolor = 'gainsboro',
                                     frameon=True, loc='upper left')
                self.axvv.legend(handles=[legCentroids, legSecCentroids],facecolor = 'gainsboro',
                                 frameon=True, loc='upper right')
            else:
                self.axvv.legend(handles=[legCentroids],facecolor = 'gainsboro',frameon=True, loc='upper right')
                if not self.dfclvp_is_empty:
                    self.axvp.legend(handles=[legCentroids],facecolor = 'gainsboro',frameon=True, loc='upper left')

        #self.fig.canvas.flush_events()
        #plt.gcf()
        #self.fig.canvas.draw()
        plt.draw()
        #self.save_plot(val)
        #plt.show()
        self.text_log.value = 'Plot actualizado.'
        #imprimir a consola
        os.write(1, b"Plot actualizado\n")

# dataframe to cluster

In [1]:
'''TO DO: en el idx_centroids los numeros de los clusters no llevan la C, en el idx_centroids_sc si
la llevan, o cambiar todo a que la lleven o que no la lleven'''

from sklearn import cluster, datasets
from scipy.spatial.distance import cdist
#crear clusters a partir de dataframes
def dataframe_to_cluster(dfvxvy, n_clusters,dfVP =None, n_subclu=0, clusters_data=None,subclusters_data=None,
                         datadir=None):
    """
        Descripcion:
        Esta funcion toma diferentes dataframes de entrada agrupa los datos por cluster.

        :param subclusters_data: data to do subclustering [wind,pow,dir]
        :param clusters_data: data to do clustering [wind,pow,dir]
        :param dfvxvy: dataframe con columnas vx y vy
        :param dfVP: dataframe con columnas de magnitud de viento y potencia
        :param n_clusters: numero de clusters
        :param n_subclu: Número de suclusters a calcular a partir de los n clusters calculados en un principio.
                  Si n_subclu=0 no se calcula ningun subcluster. Por defecto n_sub=0


        :return cl_ord: un array que contiene el número de cluster n ordenado de menor a mayor
            de las magnitudes de la velocidad de viento sin tomar en cuenta la direccion
        :return dfclvv: dataframe donde las componentes de velocidad de viento vx y vy estan
            agrupadas por cluster
        :return dfclpw: dataframe donde la potencia esta agrupada por cluster
        :return dfclvp: dataframe donde la potencia y la magnitud de viento esta agrupada por
            cluster
    """
    #check if dataframe with power values exist
    if dfVP is None:
        dfvp_is_empty = True
    else:
        dfvp_is_empty = False
    #'''- KMEANS -----------------------------------------------------------------------------------------'''
    if clusters_data=='wind':
        kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(dfvxvy)
        #buscar centroides en dataframes
        #https://stackoverflow.com/questions/42583995/get-the-centroid-row-index-from-k-means-clustering-using-sklearn
        min_dist = np.min(cdist(dfvxvy.values, kmeans.cluster_centers_, 'euclidean'), axis=1)#distancia minima a cada centroide
        Y = pd.DataFrame(min_dist, index=dfvxvy.index, columns=['PCTimeStamp'])
        Z = pd.DataFrame(kmeans.labels_, index=dfvxvy.index, columns=['cluster_ID'])
        #crea el dataframe
        PAP = pd.concat([Y,Z], axis=1)

    if clusters_data=='pow' and not dfvp_is_empty:

        kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(dfVP)
        #buscar centroides en dataframes
        min_dist = np.min(cdist(dfVP.values, kmeans.cluster_centers_, 'euclidean'), axis=1)
        Y = pd.DataFrame(min_dist, index=dfVP.index, columns=['PCTimeStamp'])
        Z = pd.DataFrame(kmeans.labels_, index=dfVP.index, columns=['cluster_ID'])
        PAP = pd.concat([Y,Z], axis=1)

    elif clusters_data=='pow' and  dfvp_is_empty:

        print('Power data is emtpy...')
        return None

    if clusters_data=='dir' and not dfvp_is_empty:
        kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(datadir)
        #buscar centroides en dataframes
        min_dist = np.min(cdist(dfVP.values, kmeans.cluster_centers_, 'euclidean'), axis=1)
        Y = pd.DataFrame(min_dist, index=dfVP.index, columns=['PCTimeStamp'])
        Z = pd.DataFrame(kmeans.labels_, index=dfVP.index, columns=['cluster_ID'])
        PAP = pd.concat([Y,Z], axis=1)

    elif clusters_data =='dir' and dfvp_is_empty:

        print('Clustering with only direction and no power data not implemented yet.')
        return None

    #ordend e los cluster por magnitud de vv
    cl_magni = np.zeros(n_clusters)

    for i in range(n_clusters):
        vx = dfvxvy.vx.values[kmeans.labels_ == i]
        vy = dfvxvy.vy.values[kmeans.labels_ == i]
        cl_magni[i] = np.mean(np.sqrt(vx**2 + vy**2))  #magnitud de la vv
    cl_ord = np.argsort(cl_magni.argsort()) #ver https://github.com/numpy/numpy/issues/8757

    grouped = PAP.groupby(['cluster_ID'])#agrupa por numero de clusters las distancias minimas
    idx_centroids = grouped.idxmin()#encuentra el indice de la distanciam minima
    #agregando columnas de velocidadd de viento y potencia al dataframe de los centroides
    vxvy=dfvxvy.loc[idx_centroids.PCTimeStamp].values
    if dfvp_is_empty:
        idx_centroids=idx_centroids.assign(vx=vxvy[:,0],vy=vxvy[:,1])
    else:
        vvpot = dfVP.loc[idx_centroids.PCTimeStamp].values
        idx_centroids=idx_centroids.assign(vx=vxvy[:,0],vy=vxvy[:,1],vViento=vvpot[:,0],Pw=vvpot[:,1])
        idx_centroids.sort_values(by='vViento',inplace=True)# ordenar por velocidad de viento


    idx_centroids.reset_index(inplace=True) #que ya no sea el cluster id el indice
    idx_centroids.index.set_names('cluster_ID_ord',inplace=True)  #ponerle el nombre al indice ordenado
    idx_centroids.index+=1 #paraque el indice empieze en uno y no exista cluster 0




    ## CREAR MULTIINDICE ---------------------------------------------------------------'''
    #CLUSTER VX,VY:
    #https://stackoverflow.com/questions/37835508/how-to-do-multi-column-from-tuples
    #nombre de las columnas del dataframe
    if dfvp_is_empty:
        colheadvv = []
        for i in range(n_clusters):
            colheadvv.append(('C' + str(i + 1), 'vx'))
            colheadvv.append(('C' + str(i + 1), 'vy'))
        dfclvv = pd.DataFrame()

        for i in range(n_clusters):
            dfclvv = pd.concat([
                dfclvv, dfvxvy.vx[kmeans.labels_ ==np.where(cl_ord ==i)[0][0]],
                dfvxvy.vy[kmeans.labels_ == np.where(cl_ord ==i)[0][0]]
            ],
                axis=1,
                ignore_index=True)
        #crear multiindice
        dfclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

    else:

        colheadvv = []
        colheadpw = []
        for i in range(n_clusters):
            colheadvv.append(('C' + str(i + 1), 'vx'))
            colheadvv.append(('C' + str(i + 1), 'vy'))
            colheadpw.append('C' + str(i + 1))
        dfclvv = pd.DataFrame()

        for i in range(n_clusters):
            dfclvv = pd.concat([
                dfclvv, dfvxvy.vx[kmeans.labels_ ==np.where(cl_ord ==i)[0][0]],
                dfvxvy.vy[kmeans.labels_ == np.where(cl_ord ==i)[0][0]]
            ],
                axis=1,
                ignore_index=True)
        #crear multiindice
        dfclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

        #CLUSTER POTENCIA:
        dfclpw = pd.DataFrame()
        for i in range(n_clusters):
            dfclpw = pd.concat([dfclpw, dfVP.Pw[kmeans.labels_ == np.where(cl_ord ==i)[0][0]]],
                               ignore_index=True,
                               axis=1)
        dfclpw.columns = colheadpw

        #CLUSTER VIENTO POTENCIA:

        colheadvp = []
        for i in range(n_clusters):
            colheadvp.append(('C' + str(i + 1), 'vViento'))
            colheadvp.append(('C' + str(i + 1), 'Pw'))
        dfclvp = pd.DataFrame()



        for i in range(n_clusters):
            dfclvp = pd.concat([
                dfclvp, dfVP.vViento[kmeans.labels_ == np.where(cl_ord ==i)[0][0]],
                dfVP.Pw[kmeans.labels_ ==np.where(cl_ord ==i)[0][0]]] ,
                axis=1,
                ignore_index=True)
        #crear multiindice
        dfclvp.columns = pd.MultiIndex.from_tuples(colheadvp)

    if clusters_data=='direccion':
        #CLUSTER direccion viento
        colheadvp = []
        for i in range(n_clusters):
            colheadvp.append(('C' + str(i + 1), 'Dir'))
            colheadvp.append(('C' + str(i + 1), 'vViento'))
        dfcldv = pd.DataFrame()
        for i in range(n_clusters):
            dfcldv = pd.concat([
                dfcldv, datadir.vViento[kmeans.labels_ ==( cl_ord ==i)],
                datadir.Dir[kmeans.labels_ == (cl_ord ==i)]],
                axis=1,ignore_index=True)
        #crear multiindice
        dfcldv.columns = pd.MultiIndex.from_tuples(colheadvp)
    '''- CALCULAR SUBCLUSTERS ------------------------------------------------------------------- '''


    if n_subclu > 0:
        scl_centroids = []
        scl_labels = []
        scl_ncentroids =[]
        idx_centroids_sc= pd.DataFrame() #va almacenar los centroides de los subclusters
        #obtener los resultados del clusterizado
        for i in range(n_clusters):
            if subclusters_data=='wind':
                dfclvvnoNA=dfclvv['C'+str(i+1)].dropna()
                #buscando centroides con los subclusters
                kmeans_sc = cluster.KMeans(n_clusters=n_subclu, random_state=0).fit(dfclvvnoNA)
                min_dist_sc = np.min(cdist(dfclvvnoNA.values, kmeans_sc.cluster_centers_, 'euclidean'), axis=1)
                Y_sc = pd.DataFrame(min_dist_sc, index=dfclvvnoNA.index, columns=['PCTimeStamp'])
                Z_sc = pd.DataFrame(kmeans_sc.labels_, index=dfclvvnoNA.index, columns=['subcluster_ID'])

            elif subclusters_data=='pow':
                dfclvpnoNA=dfclvp['C'+str(i+1)].dropna()
                #buscando centroides con los subclusters
                kmeans_sc = cluster.KMeans(n_clusters=n_subclu, random_state=0).fit(dfclvpnoNA)
                min_dist_sc = np.min(cdist(dfclvpnoNA.values, kmeans_sc.cluster_centers_, 'euclidean'), axis=1)
                Y_sc = pd.DataFrame(min_dist_sc, index=dfclvpnoNA.index, columns=['PCTimeStamp'])
                Z_sc = pd.DataFrame(kmeans_sc.labels_, index=dfclvpnoNA.index, columns=['subcluster_ID'])
            else:
                print('Todavía no programado, haciendo subclusters en potencia')
                kmeans_sc = cluster.KMeans(n_clusters=n_subclu, random_state=0).fit(dfclvp['C'+str(i+1)].dropna())

            scl_centroids.append(kmeans_sc.cluster_centers_)
            scl_labels.append(kmeans_sc.labels_)
            scl_ncentroids.append(len(scl_centroids[i]))


            PAP_sc = pd.concat([Y_sc,Z_sc], axis=1)
            #poniendo index a los centroides de los subclusters
            grouped_sc = PAP_sc.groupby(['subcluster_ID'])
            idx_cent_sc = grouped_sc.idxmin()
            idx_cent_sc.index +=1
            #agregando columnas de velocidadd de viento y potencia al dataframe de los centroides
            vxvy_sc=dfvxvy.loc[idx_cent_sc.PCTimeStamp].values
            vvpot_sc = dfVP.loc[idx_cent_sc.PCTimeStamp].values
            idx_centroids_sc=pd.concat([idx_centroids_sc, idx_cent_sc.assign(vx=vxvy_sc[:,0],vy=vxvy_sc[:,1],vViento=vvpot_sc[:,0],
                                                                             Pw=vvpot_sc[:,1], cluster_ID='C'+str(i+1))] )
        #creando el multiindex fuera del ciclo for
        # idx_centroids_sc.set_index(['cluster_ID',idx_centroids_sc.index],inplace=True)

        idx_centroids_sc.reset_index(inplace=True)
        idx_centroids_sc.sort_values(['cluster_ID','vViento'], ascending=[1,1],inplace=True)
        lst_num_sc=list(range(1,n_subclu+1))*n_clusters#crea una lista de numeros para el subcluster [1,2,3..1,2,3]
        sc_idx_list =list(map(lambda x:'SC'+str(x),lst_num_sc))#le pone las letras SC [SC1,SC2,SC3...SC1,SC2,SC3]
        idx_centroids_sc=idx_centroids_sc.assign(subcluster_ID_ord=sc_idx_list)
        idx_centroids_sc.set_index(['cluster_ID','subcluster_ID_ord'],inplace=True)

        ############  ORDENAR CENTROIDES:  ordenar el orden de aparicion segun la magnitud de la vv
        scl_magni = np.zeros([n_clusters,n_subclu])
        scl_ord = []

        for i in range(n_clusters):

            for j in range(n_subclu):

                vx = dfclvv['C' + str(i+1)].vx.dropna().values[scl_labels[i] == j]
                vy = dfclvv['C' + str(i+1)].vy.dropna().values[scl_labels[i] == j]
                scl_magni[i][j] = np.mean(np.sqrt(vx**2 + vy**2))  #magnitud de la vv

            scl_ord.append( scl_magni[i].argsort())# ver https://github.com/numpy/numpy/issues/8757

        #ORDENAR CENTROIDES
        for i in range(len(scl_centroids)):
            scl_centroids[i] = scl_centroids[i][scl_ord[i]]

        #CLUSTER VIENTO POTENCIA:
        colheadvp = []
        for i in range(n_clusters):
            for j in range(n_subclu):
                colheadvp.append(('C' + str(i + 1),'SC' + str(j + 1), 'vViento'))
                colheadvp.append(('C' + str(i + 1),'SC' + str(j + 1), 'Pw'))
        dfsclvp = pd.DataFrame()

        for i in range(n_clusters):
            for j in range(n_subclu):
                dfsclvp = pd.concat([
                    dfsclvp, dfclvp['C'+str(i+1)].dropna().vViento[scl_labels[i]  == scl_ord[i][j]],
                    dfclvp['C'+str(i+1)].dropna().Pw[scl_labels[i] == scl_ord[i][j]]
                ],
                    axis=1,
                    ignore_index=True)
        #crear multiindice
        dfsclvp.columns = pd.MultiIndex.from_tuples(colheadvp)

        #CLUSTER VX,VY:####################################################
        colheadvv = []
        for i in range(n_clusters):
            for j in range(n_subclu):
                colheadvv.append(('C' + str(i + 1),'SC' + str(j + 1), 'vx'))
                colheadvv.append(('C' + str(i + 1),'SC' + str(j + 1), 'vy'))
        dfsclvv = pd.DataFrame()

        for i in range(n_clusters):
            for j in range(n_subclu):
                dfsclvv = pd.concat([
                    dfsclvv, dfclvv['C'+str(i+1)].dropna().vx[scl_labels[i]  == scl_ord[i][j]],
                    dfclvv['C'+str(i+1)].dropna().vy[scl_labels[i] == scl_ord[i][j]]
                ],
                    axis=1,
                    ignore_index=True)
        #crear multiindice
        dfsclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

        #CLUSTER POTENCIA:
        dfsclpw = pd.DataFrame()
        colheadpw =[]
        for i in range(n_clusters):
            for j in range(n_subclu):
                colheadpw.append(('C' + str(i + 1),'SC' + str(j + 1)))

        for i in range(n_clusters):
            for j in range(n_subclu):
                dfsclpw = pd.concat([dfsclpw, dfclvp['C'+str(i+1)].Pw.dropna()[scl_labels[i] == scl_ord[i][j]]],
                                    ignore_index=True,
                                    axis=1)
        dfsclpw.columns = pd.MultiIndex.from_tuples(colheadpw)

        #### buscar centroides para la grafica vv
        #obtener nombre de niveles
        n_subclu = len(dfsclvv.columns.levels[1])
        #numero total de clusters incluidos los subclusters
        n_tot_clusters = n_subclu*n_clusters
        # solo aplica cuando hay subclusters
        lev0 = dfsclvv.columns.get_level_values(0)
        lev1 = dfsclvv.columns.get_level_values(1)
        namcl = [(lev0[i],lev1[i]) for i in range(len(lev0))]
        colnames = namcl[::2]


        return  dfsclvv,dfsclpw,dfsclvp,cl_ord,kmeans.cluster_centers_   , idx_centroids,scl_ord,scl_centroids,idx_centroids_sc

    else:# sin subclusters
        scl_ord=[]
        scl_centroids=[]
        idx_centroids_sc=[]
        if dfvp_is_empty:
            return dfclvv,[],[],cl_ord,kmeans.cluster_centers_,idx_centroids,scl_ord,scl_centroids, idx_centroids_sc
        else:
            return dfclvv,dfclpw,dfclvp,cl_ord,kmeans.cluster_centers_,idx_centroids,scl_ord,scl_centroids, idx_centroids_sc


# Eliminar clusters

In [1]:
''' FUNCION ACTUALIZADA A AGOSTO VER: https://pandas-docs.github.io/pandas-docs-travis/user_guide/advanced.html
La palabra labels cambio a codes
Changed in version 0.24.0: MultiIndex.labels has been renamed to MultiIndex.codes and MultiIndex.set_labels
to MultiIndex.set_codes.
'''
'''TODO: POR ALGUNA RAZON NO PUEDO EJECUTAR LA MISMA FUNCION DOS VECES. ES COMO SI AFECTARA MIS VARIABLES ORIGINALES'''
def del_clusters(data,cltodel,idx_cent,idx_cent_sc=None,scl_ord=None,cl_type='cluster'):
    """
    Elimina clusters, subclusters, centroides de clusters o centroides de subclusters.
        Argumentos:
            dataf: dataframes a los que se eliminaran los clusters o subclusters.
            cltodel: lista o listas de clusters o subclusters a eliminar
            sc_centroids: lista que contiene los centroides. De esta lista se eliminaran los centroides
                            a partir de cltodel.
    """
    dflist =[]
    for df in data:
        dflist.append(df.drop(cltodel,axis=1))
    #cambiar nombre por numero de centroides
    n_cl=len(df.columns.levels[0])
    n_subcl = len(df.columns.levels[1])
    #numero total de clusters incluidos los subclusters
    #sn_tot_clusters = (n_subcl*n_cl) -(len(df.columns.labels[0])-len(dflist[0].columns.labels[0]))/2
    #sn_tot_clusters = (n_subcl*n_cl) -(len(df.columns.codes[0])-len(dflist[0].columns.codes[0]))/2
    #eliminar centroides de clusters
    clnames = pd.unique(df.columns.get_level_values(0))
    lev0 = data[0].columns.get_level_values(0)
    lev1 = data[0].columns.get_level_values(1)
    namcl = [(lev0[i],lev1[i]) for i in range(len(lev0))]
    cl_sc_names = namcl[::2]
    #convertir a conjuntos
    a =set(cltodel)# clusters a eliminar
    b=set(cl_sc_names)#nombre de todos los clusters
    c=(b-a)#a b lequito a
    cent_restantes =set(sorted(set(int(item[0][1:]) for item in c)))#quito la letra C
    idx_cent_reales = set(idx_cent.index.values)
    idx_cent_todel = list(idx_cent_reales-cent_restantes)
    idx_centroids_clean=idx_cent.drop(idx_cent_todel)

    #eliminar centroides de los subclusters    
    idx_centroidssc_clean = idx_cent_sc.drop(cltodel)
  
    return dflist,idx_centroids_clean,idx_centroidssc_clean

    


# clusters a datos


In [3]:
def clust_to_data(dfv,dfvp):
    """
    Convierte el dataframe de los datos de vx,vy y potencia separados en clusters (C1-SC1-vx,vy) a datos con
    con tres columnas time,vx,vy o time,vv,pw.
    To-do:   Sigue perdiendo miles de registros, ¿porque hay registros na en el original?
    hay filas con solo nan en el original, en una prueba fueron como 4000"""
    dfv= dfv.copy()#para que no afecte el original, si lo afecta el que esta como argumento
    dfvp= dfvp.copy()
    #Quitar multiindex
    dfv.columns = dfv.columns.map(''.join)
    dfv.reset_index(inplace=True)
    #list comprehension
    lvv=[
       dfv.iloc[i].dropna().values
       for i in range(len(dfv) )      
      ]
    dfvxvy_clean=pd.DataFrame(lvv,columns=['PCTimeStamp','vx','vy'])
    dfvxvy_clean.dropna(inplace=True)#hay filas de nan y se pierden miles de datos, revisar
    
    #Quitar multiindex
    dfvp.columns = dfvp.columns.map(''.join)
    dfvp.reset_index(inplace=True)
    #list comprehension
    lvp=[
       dfvp.iloc[i].dropna().values
       for i in range(len(dfvp) )      
      ]
    dfvp_clean=pd.DataFrame(lvp,columns=['PCTimeStamp','vViento','Pw'])
    dfvp_clean.dropna(inplace=True)#hay filas de nan y se pierden miles de datos, revisar
    
    return dfvxvy_clean,dfvp_clean

datetime.datetime(2022, 3, 1, 17, 22, 53, 439969)

# daymin2date

In [1]:
import datetime
def daymin2date(year,day,hour_min):
    """

    :param year: year when data was measured
    :param day: day
    :param hour_min: hour and minute in 24 hour format without separating character, e.g. 16:37 --> 1637
    :return: timestamp in format dd/mm/YYYY HH:MM
    """
    hourmin =hour_min.zfill(4)
    hour = hourmin[:2]
    minute = hourmin[2:]
    res = datetime.datetime.strptime(year + "-" + day +" "+hour +":"+minute, "%Y-%j %H:%M").strftime("%d/%m/%Y %H:%M")
    return res

# Class kmData

In [3]:
'''
TO DO:
-en el idx_centroids los numeros de los clusters no llevan la C, en el idx_centroids_sc si
la llevan, o cambiar todo a que la lleven o que no la lleven
'''

from sklearn import cluster, datasets
from scipy.spatial.distance import cdist

class KMData:
    """
    Clase para tener un objeto con todos los datos de los clusters.
    """
    def __init__(self):
        self.kmeans_labels = None
        self.dfclvv = None
        self.cl_ord = None
        self.scl_ord= None
        self.scl_centroids = None
        self.idx_centroids = None
        self.idx_centroids_sc = None
        self.cl_centers = None
        self.dfclpw = None
        self.dfclvp= None

    #crear clusters a partir de dataframes
    def dataframe_to_cluster(self,dfvxvy, n_clusters,dfVP =None, n_sub_clusters=0, clusters_data=None,
                             subclusters_data=None, datadir=None):
        """
            Descripcion:
            Esta funcion toma diferentes dataframes de entrada agrupa los datos por cluster.

            :param subclusters_data: data to do subclustering [wind,pow,dir]
            :param clusters_data: data to do clustering [wind,pow,dir]
            :param dfvxvy: dataframe con columnas vx y vy
            :param dfVP: dataframe con columnas de magnitud de viento y potencia
            :param n_clusters: numero de clusters
            :param n_sub_clusters: Número de suclusters a calcular a partir de los n clusters calculados en un principio.
                      Si n_sub_clusters=0 no se calcula ningun subcluster. Por defecto n_sub=0


            :return cl_ord: un array que contiene el número de cluster n ordenado de menor a mayor
                de las magnitudes de la velocidad de viento sin tomar en cuenta la direccion
            :return dfclvv: dataframe donde las componentes de velocidad de viento vx y vy estan
                agrupadas por cluster
            :return dfclpw: dataframe donde la potencia esta agrupada por cluster
            :return dfclvp: dataframe donde la potencia y la magnitud de viento esta agrupada por
                cluster
        """

        #inicializacion de variables internas
        dfclpw = None
        dfclvp = None
        scl_ord = None
        scl_centroids = None
        idx_centroids_sc = None

        #check if dataframe with power values exist
        if dfVP is None:
            dfvp_is_empty = True
        else:
            dfvp_is_empty = False

        #'''- KMEANS -----------------------------------------------------------------------------------------'''
        if clusters_data=='wind':
            kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(dfvxvy)
            #buscar centroides en dataframes
            #https://stackoverflow.com/questions/42583995/get-the-centroid-row-index-from-k-means-clustering-using-sklearn
            min_dist = np.min(cdist(dfvxvy.values, kmeans.cluster_centers_, 'euclidean'), axis=1)#distancia minima a cada centroide
            Y = pd.DataFrame(min_dist, index=dfvxvy.index, columns=['PCTimeStamp'])
            Z = pd.DataFrame(kmeans.labels_, index=dfvxvy.index, columns=['cluster_ID'])
            #crea el dataframe
            PAP = pd.concat([Y,Z], axis=1)

        if clusters_data=='pow' and not dfvp_is_empty:

            kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(dfVP)
            #buscar centroides en dataframes
            min_dist = np.min(cdist(dfVP.values, kmeans.cluster_centers_, 'euclidean'), axis=1)
            Y = pd.DataFrame(min_dist, index=dfVP.index, columns=['PCTimeStamp'])
            Z = pd.DataFrame(kmeans.labels_, index=dfVP.index, columns=['cluster_ID'])
            PAP = pd.concat([Y,Z], axis=1)

        elif clusters_data=='pow' and  dfvp_is_empty:

            print('Power data is emtpy...')
            return None

        if clusters_data=='dir' and not dfvp_is_empty:
            kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(datadir)
            #buscar centroides en dataframes
            min_dist = np.min(cdist(dfVP.values, kmeans.cluster_centers_, 'euclidean'), axis=1)
            Y = pd.DataFrame(min_dist, index=dfVP.index, columns=['PCTimeStamp'])
            Z = pd.DataFrame(kmeans.labels_, index=dfVP.index, columns=['cluster_ID'])
            PAP = pd.concat([Y,Z], axis=1)

        elif clusters_data =='dir' and dfvp_is_empty:

            print('Clustering with only direction and no power data not implemented yet.')
            return None

        #ordend e los cluster por magnitud de vv
        cl_magni = np.zeros(n_clusters)

        for i in range(n_clusters):
            vx = dfvxvy.vx.values[kmeans.labels_ == i]
            vy = dfvxvy.vy.values[kmeans.labels_ == i]
            cl_magni[i] = np.mean(np.sqrt(vx**2 + vy**2))  #magnitud de la vv
        cl_ord = np.argsort(cl_magni.argsort()) #ver https://github.com/numpy/numpy/issues/8757

        grouped = PAP.groupby(['cluster_ID'])#agrupa por numero de clusters las distancias minimas
        idx_centroids = grouped.idxmin()#encuentra el indice de la distanciam minima
        #agregando columnas de velocidadd de viento y potencia al dataframe de los centroides
        vxvy=dfvxvy.loc[idx_centroids.PCTimeStamp].values
        if dfvp_is_empty:
            idx_centroids=idx_centroids.assign(vx=vxvy[:,0],vy=vxvy[:,1])
        else:
            vvpot = dfVP.loc[idx_centroids.PCTimeStamp].values
            idx_centroids=idx_centroids.assign(vx=vxvy[:,0],vy=vxvy[:,1],vViento=vvpot[:,0],Pw=vvpot[:,1])
            idx_centroids.sort_values(by='vViento',inplace=True)# ordenar por velocidad de viento


        idx_centroids.reset_index(inplace=True) #que ya no sea el cluster id el indice
        idx_centroids.index.set_names('cluster_ID_ord',inplace=True)  #ponerle el nombre al indice ordenado
        idx_centroids.index+=1 #paraque el indice empieze en uno y no exista cluster 0




        ## CREAR MULTIINDICE ---------------------------------------------------------------'''
        #CLUSTER VX,VY:
        #https://stackoverflow.com/questions/37835508/how-to-do-multi-column-from-tuples
        #nombre de las columnas del dataframe
        if dfvp_is_empty:
            colheadvv = []
            for i in range(n_clusters):
                colheadvv.append(('C' + str(i + 1), 'vx'))
                colheadvv.append(('C' + str(i + 1), 'vy'))
            dfclvv = pd.DataFrame()

            for i in range(n_clusters):
                dfclvv = pd.concat([
                    dfclvv, dfvxvy.vx[kmeans.labels_ ==np.where(cl_ord ==i)[0][0]],
                    dfvxvy.vy[kmeans.labels_ == np.where(cl_ord ==i)[0][0]]
                ],
                    axis=1,
                    ignore_index=True)
            #crear multiindice
            dfclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

        else:

            colheadvv = []
            colheadpw = []
            for i in range(n_clusters):
                colheadvv.append(('C' + str(i + 1), 'vx'))
                colheadvv.append(('C' + str(i + 1), 'vy'))
                colheadpw.append('C' + str(i + 1))
            dfclvv = pd.DataFrame()

            for i in range(n_clusters):
                dfclvv = pd.concat([
                    dfclvv, dfvxvy.vx[kmeans.labels_ ==np.where(cl_ord ==i)[0][0]],
                    dfvxvy.vy[kmeans.labels_ == np.where(cl_ord ==i)[0][0]]
                ],
                    axis=1,
                    ignore_index=True)
            #crear multiindice
            dfclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

            #CLUSTER POTENCIA:
            dfclpw = pd.DataFrame()
            for i in range(n_clusters):
                dfclpw = pd.concat([dfclpw, dfVP.Pw[kmeans.labels_ == np.where(cl_ord ==i)[0][0]]],
                                   ignore_index=True,
                                   axis=1)
            dfclpw.columns = colheadpw

            #CLUSTER VIENTO POTENCIA:

            colheadvp = []
            for i in range(n_clusters):
                colheadvp.append(('C' + str(i + 1), 'vViento'))
                colheadvp.append(('C' + str(i + 1), 'Pw'))
            dfclvp = pd.DataFrame()

            for i in range(n_clusters):
                dfclvp = pd.concat([
                    dfclvp, dfVP.vViento[kmeans.labels_ == np.where(cl_ord ==i)[0][0]],
                    dfVP.Pw[kmeans.labels_ ==np.where(cl_ord ==i)[0][0]]] ,
                    axis=1,
                    ignore_index=True)
            #crear multiindice
            dfclvp.columns = pd.MultiIndex.from_tuples(colheadvp)

        if clusters_data=='direccion':
            #CLUSTER direccion viento
            colheadvp = []
            for i in range(n_clusters):
                colheadvp.append(('C' + str(i + 1), 'Dir'))
                colheadvp.append(('C' + str(i + 1), 'vViento'))
            dfcldv = pd.DataFrame()
            for i in range(n_clusters):
                dfcldv = pd.concat([
                    dfcldv, datadir.vViento[kmeans.labels_ ==( cl_ord ==i)],
                    datadir.Dir[kmeans.labels_ == (cl_ord ==i)]],
                    axis=1,ignore_index=True)
            #crear multiindice
            dfcldv.columns = pd.MultiIndex.from_tuples(colheadvp)
        '''- CALCULAR SUBCLUSTERS ------------------------------------------------------------------- '''
        if n_sub_clusters > 0:
            scl_centroids = []
            scl_labels = []
            scl_ncentroids =[]
            idx_centroids_sc= pd.DataFrame() #va almacenar los centroides de los subclusters
            #obtener los resultados del clusterizado
            for i in range(n_clusters):
                if subclusters_data=='wind':
                    dfclvvnoNA=dfclvv['C'+str(i+1)].dropna()
                    #buscando centroides con los subclusters
                    kmeans_sc = cluster.KMeans(n_clusters=n_sub_clusters, random_state=0).fit(dfclvvnoNA)
                    min_dist_sc = np.min(cdist(dfclvvnoNA.values, kmeans_sc.cluster_centers_, 'euclidean'), axis=1)
                    Y_sc = pd.DataFrame(min_dist_sc, index=dfclvvnoNA.index, columns=['PCTimeStamp'])
                    Z_sc = pd.DataFrame(kmeans_sc.labels_, index=dfclvvnoNA.index, columns=['subcluster_ID'])

                elif subclusters_data=='pow':
                    dfclvpnoNA=dfclvp['C'+str(i+1)].dropna()
                    #buscando centroides con los subclusters
                    kmeans_sc = cluster.KMeans(n_clusters=n_sub_clusters, random_state=0).fit(dfclvpnoNA)
                    min_dist_sc = np.min(cdist(dfclvpnoNA.values, kmeans_sc.cluster_centers_, 'euclidean'), axis=1)
                    Y_sc = pd.DataFrame(min_dist_sc, index=dfclvpnoNA.index, columns=['PCTimeStamp'])
                    Z_sc = pd.DataFrame(kmeans_sc.labels_, index=dfclvpnoNA.index, columns=['subcluster_ID'])
                else:
                    print('Todavía no programado, haciendo subclusters en potencia')
                    kmeans_sc = cluster.KMeans(n_clusters=n_sub_clusters, random_state=0).fit(dfclvp['C'+str(i+1)].dropna())

                scl_centroids.append(kmeans_sc.cluster_centers_)
                scl_labels.append(kmeans_sc.labels_)
                scl_ncentroids.append(len(scl_centroids[i]))


                PAP_sc = pd.concat([Y_sc,Z_sc], axis=1)
                #poniendo index a los centroides de los subclusters
                grouped_sc = PAP_sc.groupby(['subcluster_ID'])
                idx_cent_sc = grouped_sc.idxmin()
                idx_cent_sc.index +=1
                #agregando columnas de velocidadd de viento y potencia al dataframe de los centroides
                vxvy_sc=dfvxvy.loc[idx_cent_sc.PCTimeStamp].values
                vvpot_sc = dfVP.loc[idx_cent_sc.PCTimeStamp].values
                idx_centroids_sc=pd.concat([idx_centroids_sc, idx_cent_sc.assign(vx=vxvy_sc[:,0],vy=vxvy_sc[:,1],vViento=vvpot_sc[:,0],
                                                                                 Pw=vvpot_sc[:,1], cluster_ID='C'+str(i+1))] )
            #creando el multiindex fuera del ciclo for
            # idx_centroids_sc.set_index(['cluster_ID',idx_centroids_sc.index],inplace=True)

            idx_centroids_sc.reset_index(inplace=True)
            idx_centroids_sc.sort_values(['cluster_ID','vViento'], ascending=[1,1],inplace=True)
            lst_num_sc=list(range(1,n_sub_clusters+1))*n_clusters#crea una lista de numeros para el subcluster [1,2,3..1,2,3]
            sc_idx_list =list(map(lambda x:'SC'+str(x),lst_num_sc))#le pone las letras SC [SC1,SC2,SC3...SC1,SC2,SC3]
            idx_centroids_sc=idx_centroids_sc.assign(subcluster_ID_ord=sc_idx_list)
            idx_centroids_sc.set_index(['cluster_ID','subcluster_ID_ord'],inplace=True)

            ############  ORDENAR CENTROIDES:  ordenar el orden de aparicion segun la magnitud de la vv
            scl_magni = np.zeros([n_clusters,n_sub_clusters])
            scl_ord = []

            for i in range(n_clusters):

                for j in range(n_sub_clusters):

                    vx = dfclvv['C' + str(i+1)].vx.dropna().values[scl_labels[i] == j]
                    vy = dfclvv['C' + str(i+1)].vy.dropna().values[scl_labels[i] == j]
                    scl_magni[i][j] = np.mean(np.sqrt(vx**2 + vy**2))  #magnitud de la vv

                scl_ord.append( scl_magni[i].argsort())# ver https://github.com/numpy/numpy/issues/8757

            #ORDENAR CENTROIDES
            for i in range(len(scl_centroids)):
                scl_centroids[i] = scl_centroids[i][scl_ord[i]]

            #CLUSTER VIENTO POTENCIA:
            colheadvp = []
            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    colheadvp.append(('C' + str(i + 1),'SC' + str(j + 1), 'vViento'))
                    colheadvp.append(('C' + str(i + 1),'SC' + str(j + 1), 'Pw'))
            dfsclvp = pd.DataFrame()

            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    dfsclvp = pd.concat([
                        dfsclvp, dfclvp['C'+str(i+1)].dropna().vViento[scl_labels[i]  == scl_ord[i][j]],
                        dfclvp['C'+str(i+1)].dropna().Pw[scl_labels[i] == scl_ord[i][j]]
                    ],
                        axis=1,
                        ignore_index=True)
            #crear multiindice
            dfsclvp.columns = pd.MultiIndex.from_tuples(colheadvp)

            #CLUSTER VX,VY:####################################################
            colheadvv = []
            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    colheadvv.append(('C' + str(i + 1),'SC' + str(j + 1), 'vx'))
                    colheadvv.append(('C' + str(i + 1),'SC' + str(j + 1), 'vy'))
            dfsclvv = pd.DataFrame()

            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    dfsclvv = pd.concat([
                        dfsclvv, dfclvv['C'+str(i+1)].dropna().vx[scl_labels[i]  == scl_ord[i][j]],
                        dfclvv['C'+str(i+1)].dropna().vy[scl_labels[i] == scl_ord[i][j]]
                    ],
                        axis=1,
                        ignore_index=True)
            #crear multiindice
            dfsclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

            #CLUSTER POTENCIA:
            dfsclpw = pd.DataFrame()
            colheadpw =[]
            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    colheadpw.append(('C' + str(i + 1),'SC' + str(j + 1)))

            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    dfsclpw = pd.concat([dfsclpw, dfclvp['C'+str(i+1)].Pw.dropna()[scl_labels[i] == scl_ord[i][j]]],
                                        ignore_index=True,
                                        axis=1)
            dfsclpw.columns = pd.MultiIndex.from_tuples(colheadpw)

            #### buscar centroides para la grafica vv
            #obtener nombre de niveles
            n_sub_clusters = len(dfsclvv.columns.levels[1])
            #numero total de clusters incluidos los subclusters
            n_tot_clusters = n_sub_clusters*n_clusters
            # solo aplica cuando hay subclusters
            lev0 = dfsclvv.columns.get_level_values(0)
            lev1 = dfsclvv.columns.get_level_values(1)
            namcl = [(lev0[i],lev1[i]) for i in range(len(lev0))]
            colnames = namcl[::2]



        # llenado de datos
        self.kmeans_labels = kmeans.labels_
        self.comp_vel = dfclvv
        self.cl_ord = cl_ord
        self.idx_centroids = idx_centroids
        self.cl_centers = kmeans.cluster_centers_
        #self.dfclpw = dfclpw
        self.dfclvp= dfclvp

        self.scl_ord= scl_ord
        self.scl_centroids = scl_centroids
        self.idx_centroids_sc = idx_centroids_sc
       